In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import cv2
import re
# import os
import glob
# import re
# import numpy as np
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
import tifffile as tiff
# import os
# import numpy as np
# import glob
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
# import rasterio

def load_image(filepath):
    # Load the image using PIL
    image = np.array(Image.open(filepath).convert('L'))
    
    # Apply Otsu's thresholding
    t, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Convert to binary (0 and 1)
    binary_image = (binary_image > 0).astype(np.uint8)
    return binary_image

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection) / (np.sum(y_true) + np.sum(y_pred))

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)

# Paths to the folders containing the predictions and ground truths


In [2]:
# gt_folder = './GEE_Masks/GEE_resized/'
# pred_folder = './GEE_Output/Adversarial'

# # List all files in the prediction folder
# # pred_files = [f for f in os.listdir(pred_folder) if f.endswith('.tif')]

# # Initialize metrics



In [3]:
# # Lists to store metrics
# dice_scores = []
# iou_scores = []
# pixel_accuracies = []
# precisions = []
# recalls = []
# f1_scores = []
# specificities = []

# for i in range(0, 1263):  # Adjust the range based on the number of files
#     gt_filename = f'NDWI_Mask_{i}_resized.tif'
#     pred_filename = f'dense_Predicted_Image_{i}.tif'
    
#     gt_path = os.path.join(gt_folder, gt_filename)
#     pred_path = os.path.join(pred_folder, pred_filename)
    
#     # Load the prediction and ground truth images
#     pred_image = load_image(pred_path)
#     gt_image = load_image(gt_path)
    
#     # Flatten the images
#     pred_flat = pred_image.flatten()
#     gt_flat = gt_image.flatten()
    
#     # Calculate metrics
#     dice = dice_coefficient(gt_flat, pred_flat)
#     iou = jaccard_score(gt_flat, pred_flat, average='macro')
#     accuracy = pixel_accuracy(gt_flat, pred_flat)
#     precision = precision_score(gt_flat, pred_flat, average='macro', zero_division=0)
#     recall = recall_score(gt_flat, pred_flat, average='macro')
#     f1 = f1_score(gt_flat, pred_flat, average='macro')
#     spec = specificity(gt_flat, pred_flat)
    
#     # Store metrics
#     dice_scores.append(dice)
#     iou_scores.append(iou)
#     pixel_accuracies.append(accuracy)
#     precisions.append(precision)
#     recalls.append(recall)
#     f1_scores.append(f1)
#     specificities.append(spec)
    
#     # Print metrics for the current image
#     print(f"Metrics for {gt_filename} and {pred_filename}:")
#     print(f"  Dice Coefficient: {dice:.4f}")
#     print(f"  IoU: {iou:.4f}")
#     print(f"  Pixel Accuracy: {accuracy:.4f}")
#     print(f"  Precision: {precision:.4f}")
#     print(f"  Recall: {recall:.4f}")
#     print(f"  F1 Score: {f1:.4f}")
#     print(f"  Specificity: {spec:.4f}")
#     print()

# # Calculate and print average metrics
# average_dice = np.mean(dice_scores)
# average_iou = np.mean(iou_scores)
# average_accuracy = np.mean(pixel_accuracies)
# average_precision = np.mean(precisions)
# average_recall = np.mean(recalls)
# average_f1 = np.mean(f1_scores)
# average_specificity = np.mean(specificities)

# print("Average metrics:")
# print(f"  Average Dice Coefficient: {average_dice:.4f}")
# print(f"  Average IoU: {average_iou:.4f}")
# print(f"  Average Pixel Accuracy: {average_accuracy:.4f}")
# print(f"  Average Precision: {average_precision:.4f}")
# print(f"  Average Recall: {average_recall:.4f}")
# print(f"  Average F1 Score: {average_f1:.4f}")
# print(f"  Average Specificity: {average_specificity:.4f}")

# # Model's overall accuracy
# print(f"Model's Overall Accuracy: {average_accuracy:.4f}")

In [4]:
# # Print average metrics
# print("Average Metrics:")
# print(f"  Average Dice Coefficient: {np.mean(dice_scores):.4f}")
# print(f"  Average IoU: {np.mean(iou_scores):.4f}")
# print(f"  Average Pixel Accuracy: {np.mean(pixel_accuracies):.4f}")
# print(f"  Average Precision: {np.mean(precisions):.4f}")
# print(f"  Average Recall: {np.mean(recalls):.4f}")
# print(f"  Average F1 Score: {np.mean(f1_scores):.4f}")
# print(f"  Average Specificity: {np.mean(specificities):.4f}")

In [2]:
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)


# Directories for ground truth and prediction masks
gt_folder = './GEE_Masks/GEE_resized/'
pred_folder = './GEE_Output/Adversarial/20_Epoch_Small'

# Initialize metric lists
dice_scores = []
iou_scores = []
pixel_accuracies = []
precisions = []
recalls = []
f1_scores = []
specificities = []

# Get all prediction files
pred_files = glob.glob(os.path.join(pred_folder, 'dense_*.tif'))

for pred_file in pred_files:
    # Extract the number 'i' from the prediction filename
    match = re.search(r'dense_(\d+).tif', os.path.basename(pred_file))
    if match:
        i = match.group(1)
        gt_filename = f'NDWI_Mask_{i}_resized.tif'
        gt_path = os.path.join(gt_folder, gt_filename)
        
        # Check if the ground truth file exists
        if os.path.exists(gt_path):
            # Load the prediction and ground truth images
            pred_image = load_image(pred_file)
            gt_image = load_image(gt_path)
            print(np.max(pred_image))
            print(np.min(pred_image))
            # Flatten the images
            pred_flat = pred_image.flatten()
            gt_flat = gt_image.flatten()
            
            # Calculate metrics
            dice = dice_coefficient(gt_flat, pred_flat)
            iou = jaccard_score(gt_flat, pred_flat, average='macro')
            accuracy = pixel_accuracy(gt_flat, pred_flat)
            precision = precision_score(gt_flat, pred_flat, average='macro', zero_division=0)
            recall = recall_score(gt_flat, pred_flat, average='macro')
            f1 = f1_score(gt_flat, pred_flat, average='macro')
            spec = specificity(gt_flat, pred_flat)
            
            # Store metrics
            dice_scores.append(dice)
            iou_scores.append(iou)
            pixel_accuracies.append(accuracy)
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1)
            specificities.append(spec)
            
            # Print metrics for the current image
            print(f"Metrics for {gt_filename} and {pred_file}:")
            print(f"  Dice Coefficient: {dice:.4f}")
            print(f"  IoU: {iou:.4f}")
            print(f"  Pixel Accuracy: {accuracy:.4f}")
            print(f"  Precision: {precision:.4f}")
            print(f"  Recall: {recall:.4f}")
            print(f"  F1 Score: {f1:.4f}")
            print(f"  Specificity: {spec:.4f}")
            print()
        else:
            print(f"Ground truth file {gt_path} not found for prediction file {pred_file}.")

# Calculate and print average metrics
average_dice = np.mean(dice_scores)
average_iou = np.mean(iou_scores)
average_accuracy = np.mean(pixel_accuracies)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1 = np.mean(f1_scores)
average_specificity = np.mean(specificities)

print("Average Metrics:")
print(f"  Dice Coefficient: {average_dice:.4f}")
print(f"  IoU: {average_iou:.4f}")
print(f"  Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Precision: {average_precision:.4f}")
print(f"  Recall: {average_recall:.4f}")
print(f"  F1 Score: {average_f1:.4f}")
print(f"  Specificity: {average_specificity:.4f}")

1
0
Metrics for NDWI_Mask_10_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_10.tif:
  Dice Coefficient: 0.6591
  IoU: 0.6169
  Pixel Accuracy: 0.7937
  Precision: 0.8503
  Recall: 0.7435
  F1 Score: 0.7556
  Specificity: 0.9820

1
0
Metrics for NDWI_Mask_1008_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_1008.tif:
  Dice Coefficient: 0.8498
  IoU: 0.8578
  Pixel Accuracy: 0.9782
  Precision: 0.9697
  Recall: 0.8798
  F1 Score: 0.9191
  Specificity: 0.9972

1
0
Metrics for NDWI_Mask_1010_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_1010.tif:
  Dice Coefficient: 0.9887
  IoU: 0.9832
  Pixel Accuracy: 0.9924
  Precision: 0.9942
  Recall: 0.9889
  F1 Score: 0.9915
  Specificity: 0.9998

1
0
Metrics for NDWI_Mask_1012_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_1012.tif:
  Dice Coefficient: 0.8119
  IoU: 0.8296
  Pixel Accuracy: 0.9770
  Precision: 0.9118
  Recall: 0.8886
  F1 Score: 0.8998
  Specificity: 0.9897

1
0
Metrics 

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_418_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_418.tif:
  Dice Coefficient: 0.1111
  IoU: 0.5000
  Pixel Accuracy: 1.0000
  Precision: 0.5000
  Recall: 0.5000
  F1 Score: 0.5000
  Specificity: 1.0000

0
0
Metrics for NDWI_Mask_429_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_429.tif:
  Dice Coefficient: 1.0000
  IoU: 1.0000
  Pixel Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1 Score: 1.0000
  Specificity: 1.0000

1
0
Metrics for NDWI_Mask_43_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_43.tif:
  Dice Coefficient: 0.9774
  IoU: 0.9704
  Pixel Accuracy: 0.9887
  Precision: 0.9872
  Recall: 0.9827
  F1 Score: 0.9849
  Specificity: 0.9948

1
0
Metrics for NDWI_Mask_44_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_44.tif:
  Dice Coefficient: 0.9822
  IoU: 0.9817
  Pixel Accuracy: 0.9985
  Precision: 0.9959
  Recall: 0.9856
  F1 Score: 0.9907
  Specificity: 0.9997

1
0
Metrics for NDWI_Mas

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_810_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_810.tif:
  Dice Coefficient: 0.0001
  IoU: 0.4848
  Pixel Accuracy: 0.9697
  Precision: 0.5000
  Recall: 0.4848
  F1 Score: 0.4923
  Specificity: 0.9697

1
0
Metrics for NDWI_Mask_814_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_814.tif:
  Dice Coefficient: 0.7886
  IoU: 0.7693
  Pixel Accuracy: 0.9071
  Precision: 0.8377
  Recall: 0.9052
  F1 Score: 0.8645
  Specificity: 0.9083

1
0
Metrics for NDWI_Mask_837_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_837.tif:
  Dice Coefficient: 0.7170
  IoU: 0.7065
  Pixel Accuracy: 0.8769
  Precision: 0.7895
  Recall: 0.8734
  F1 Score: 0.8192
  Specificity: 0.8789

1
0
Metrics for NDWI_Mask_838_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_838.tif:
  Dice Coefficient: 0.3610
  IoU: 0.5435
  Pixel Accuracy: 0.8715
  Precision: 0.6134
  Recall: 0.7691
  F1 Score: 0.6448
  Specificity: 0.8843

1
0
Metrics for NDWI

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_883_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_883.tif:
  Dice Coefficient: 0.0094
  IoU: 0.3154
  Pixel Accuracy: 0.6267
  Precision: 0.5020
  Recall: 0.7091
  F1 Score: 0.3897
  Specificity: 0.6263

1
0
Metrics for NDWI_Mask_890_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_890.tif:
  Dice Coefficient: 0.0001
  IoU: 0.4673
  Pixel Accuracy: 0.9346
  Precision: 0.4997
  Recall: 0.4675
  F1 Score: 0.4831
  Specificity: 0.9350

1
0
Metrics for NDWI_Mask_893_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_893.tif:
  Dice Coefficient: 0.7515
  IoU: 0.7563
  Pixel Accuracy: 0.9214
  Precision: 0.9193
  Recall: 0.8118
  F1 Score: 0.8524
  Specificity: 0.9867

1
0
Metrics for NDWI_Mask_9_resized.tif and ./GEE_Output/Adversarial/20_Epoch_Small\dense_9.tif:
  Dice Coefficient: 0.8944
  IoU: 0.8719
  Pixel Accuracy: 0.9489
  Precision: 0.9448
  Recall: 0.9179
  F1 Score: 0.9303
  Specificity: 0.9806

1
0
Metrics for NDWI_Mas

In [3]:
print("Average metrics:")
print(f"  Average Dice Coefficient: {average_dice:.4f}")
print(f"  Average IoU: {average_iou:.4f}")
print(f"  Average Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Average Precision: {average_precision:.4f}")
print(f"  Average Recall: {average_recall:.4f}")
print(f"  Average F1 Score: {average_f1:.4f}")
print(f"  Average Specificity: {average_specificity:.4f}")

# Model's overall accuracy
print(f"Model's Overall Accuracy: {average_accuracy:.4f}")

Average metrics:
  Average Dice Coefficient: 0.7599
  Average IoU: 0.8151
  Average Pixel Accuracy: 0.9528
  Average Precision: 0.8878
  Average Recall: 0.8610
  Average F1 Score: 0.8601
  Average Specificity: 0.9728
Model's Overall Accuracy: 0.9528
